#### KNN

In [11]:
# pip install scikit-surprise

In [39]:
import ast
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

In [40]:
## 과거 데이터 로드
# 파일 경로 설정
file_path = '../train_data/Data5_train_final.csv'  # 파일 경로를 적절히 변경하세요

# pd.read_csv와 반복문을 사용하여 파일을 나누어 읽기
chunks = pd.read_csv(file_path, encoding='utf-8', chunksize=100)

# past_data라는 빈 데이터프레임 생성
past_data = pd.DataFrame()

for chunk in chunks:
    # 각 청크에 대한 처리를 여기에 추가
    # 예를 들어, 원하는 계산 수행 또는 데이터 전처리 등을 수행

    # past_data에 현재 청크를 추가
    past_data = pd.concat([past_data, chunk], ignore_index=True)

# past_data를 사용하여 나머지 작업 수행
print(past_data.head())  # past_data의 처음 몇 행을 출력하는 예시

  musical_id   title                                         poster_url genre  \
0       3885    실연남녀  http://ticketimage.interpark.com/PlayDictionar...    창작   
1       3635  김종욱 찾기  http://ticketimage.interpark.com/PlayDictionar...    창작   
2       4829     첫사랑  http://ticketimage.interpark.com/PlayDictionar...    창작   
3       3322     미라클  http://ticketimage.interpark.com/PlayDictionar...    창작   
4       4424     대장금  http://ticketimage.interpark.com/PlayDictionar...    창작   

                      date  start_date    end_date     location  \
0  2007/10/13 ~ 2007/12/09  2007-10-13  2007-12-09   대학로 티오엠 1관   
1  2006/12/12 ~ 2007/04/08  2006-12-12  2007-04-08   JTN 아트홀 1관   
2  2007/03/27 ~ 2007/06/17  2007-03-27  2007-06-17   대학로 티오엠 1관   
3  2007/06/01 ~ 2013/02/17  2007-06-01  2013-02-17   미라클 씨어터 1관   
4  2007/05/26 ~ 2007/06/17  2007-05-26  2007-06-17  예술의전당 오페라극장   

                                              actors age_rating  ...  \
0                      엄기준,신성록,강지우,양소민

In [41]:
# musical_id를 기준으로 중복된 행 찾기
duplicate_rows = past_data[past_data.duplicated(subset='musical_id', keep=False)]
# musical_id를 기준으로 중복된 행 중 첫 번째 행만 남기고 나머지 중복 행 삭제
past_data.drop_duplicates(subset='musical_id', keep='first', inplace=True)

In [42]:
#8883 rows
past_data

,musical_id,title,poster_url,genre,date,start_date,end_date,location,actors,age_rating,...,describe,synopsis,synopsis_clear,tokenized_data,synopsis_vector,synopsis_numpy,synopsis_numpy_scale,tag1,tag2,tag3
0,3885,실연남녀,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/10/13 ~ 2007/12/09,2007-10-13,2007-12-09,대학로 티오엠 1관,"엄기준,신성록,강지우,양소민,한애리,변우민,성민...",7세이상,...,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"['조폭', '형제', '경영', '아트', '퓨쳐', '산장', '이', '개월'...",b'\x0e\x11u<\xdeM1=\xa1kX=\xd11\x06\xbe?\x03\x...,[ 0.01495768 0.04328715 0.05283702 -0.131049...,"[-0.6805413961410522, 0.3137694299221039, -0.2...",사람,사랑,슬픔
1,3635,김종욱 찾기,http://ticketimage.interpark.com/PlayDictionar...,창작,2006/12/12 ~ 2007/04/08,2006-12-12,2007-04-08,JTN 아트홀 1관,"신성록,전병욱,오나라,안유진,원종환,김재범,김지현...",만 13세이상,...,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"['첫사랑', '나선', '여자', '첫사랑', '남자', '티격태격', '로맨틱'...",b'\xddC\xf5=I@Z=%\xfd\x8b\xbc\xd8\x03==\xd6q\x...,[ 0.11975835 0.05328396 -0.01708848 0.046146...,"[0.03893159702420235, 0.3932909667491913, -0.7...",사랑,오페라,NaN
2,4829,첫사랑,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/03/27 ~ 2007/06/17,2007-03-27,2007-06-17,대학로 티오엠 1관,"조정석,홍광호,해이,전경수,이정섭,김성기,김인수...",만 6세이상,...,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"['시간', '조금', '수', '있다', '적', '동네', '해수', '오누이'...","b""S\xa2\xa0<\xeb\xad\xd5<\xd6\x93\xbe\xbd\x01\...",[ 0.01960865 0.02608391 -0.09305541 -0.089630...,"[-0.6486117243766785, 0.176923006772995, -1.25...",오페라,NaN,NaN
3,3322,미라클,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/06/01 ~ 2013/02/17,2007-06-01,2013-02-17,미라클 씨어터 1관,"신현묵,조호균,김고운,원혜지,남궁인,윤도훈,김학규...",만 12세이상,...,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"['인기', '그룹', '핫바', '멤버', '가수', '희다', '은', '날',...",b'^\xb5D>\xb63\xad=P\x8e >.\x97\x02\xbe\xee+\x...,[ 1.92098111e-01 8.45712870e-02 1.56792879e-...,"[0.5355554223060608, 0.6421719193458557, 0.540...",오페라,NaN,NaN
4,4424,대장금,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/05/26 ~ 2007/06/17,2007-05-26,2007-06-17,예술의전당 오페라극장,"최보영,안유진,김우형,김소현,이태원,류수화,손광업,한애리,한성식,김나윤,김일우,최윤...",만 7세이상,...,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"['어머니', '유언', '위해', '궁', '생각', '시가', '장금', '것'...",b'@\x8a\xa8=\x8e\xd7\xe0=\x1e\x10\x9b=N%\xf6\x...,[ 0.08229494 0.10978614 0.07571433 -0.120188...,"[-0.21826054155826569, 0.84274822473526, -0.04...",사랑,오페라,아이
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11982,150211,브레멘 음악대 부산,http://ticketimage.interpark.com/PlayDictionar...,어린이가족,2019/11/16 ~ 2019/11/17,2019-11-16,2019-11-17,신세계센텀시티 문화홀9층,NaN,24개월이상,...,NaN,집에서 키워지던 동물들이 주인의 천대를 받고쫓겨나거나 스스로의 자유를 위해 집을 나...,집 동물 주인 천대 스스로 자유 위해 집 친구 멋지다 악기 음악회 만들기 즐겁다 시...,"['집', '동물', '주인', '천대', '스스로', '자유', '위해', '집'...","b'e#^\xbc\xc5o\x10\xbd\x07,\x02>\xa4\xd4\x1c>\...",[-1.35582434e-02 -3.52628417e-02 1.27121076e-...,"[-0.8763076663017273, -0.3110712766647339, 0.3...",친구,예술,NaN
11983,150242,헤이지니 럭키강이 2 청주,http://ticketimage.interpark.com/PlayDictionar...,어린이가족,2019/12/28 ~ 2019/12/29,2019-12-28,2019-12-29,청주예술의전당 대공연장,NaN,24개월이상,...,NaN,딩동 마법의 세계로 가야할 택배상자가 지니강이 집으로 배달되었다신비로운 마법의 열쇠...,딩동 마법 세계 가야 택배 상자 지니 강 집 배달 신비롭다 마법 열쇠 지니 강 비밀...,"['딩동', '마법', '세계', '가야', '택배', '상자', '지니', '강'...","b""\xe1uF=`\x1c\xb3<\x01\x92\x93>\xfc&~\xbe|\xa...",[ 0.04845226 0.02186412 0.2882233 -0.248195...,"[-0.4505958557128906, 0.14335589110851288, 1.4...",오페라,마법,NaN
11984,150349,난타 대구,http://ticketimage.interpark.com/PlayDictionar...,퍼포먼스,2019/12/20 ~ 201

In [43]:
# 'tag1' 열에 NaN 값을 가진 행을 삭제
past_data = past_data.dropna(subset=['synopsis_numpy_scale'], axis=0)
past_data

,musical_id,title,poster_url,genre,date,start_date,end_date,location,actors,age_rating,...,describe,synopsis,synopsis_clear,tokenized_data,synopsis_vector,synopsis_numpy,synopsis_numpy_scale,tag1,tag2,tag3
0,3885,실연남녀,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/10/13 ~ 2007/12/09,2007-10-13,2007-12-09,대학로 티오엠 1관,"엄기준,신성록,강지우,양소민,한애리,변우민,성민...",7세이상,...,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"['조폭', '형제', '경영', '아트', '퓨쳐', '산장', '이', '개월'...",b'\x0e\x11u<\xdeM1=\xa1kX=\xd11\x06\xbe?\x03\x...,[ 0.01495768 0.04328715 0.05283702 -0.131049...,"[-0.6805413961410522, 0.3137694299221039, -0.2...",사람,사랑,슬픔
1,3635,김종욱 찾기,http://ticketimage.interpark.com/PlayDictionar...,창작,2006/12/12 ~ 2007/04/08,2006-12-12,2007-04-08,JTN 아트홀 1관,"신성록,전병욱,오나라,안유진,원종환,김재범,김지현...",만 13세이상,...,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"['첫사랑', '나선', '여자', '첫사랑', '남자', '티격태격', '로맨틱'...",b'\xddC\xf5=I@Z=%\xfd\x8b\xbc\xd8\x03==\xd6q\x...,[ 0.11975835 0.05328396 -0.01708848 0.046146...,"[0.03893159702420235, 0.3932909667491913, -0.7...",사랑,오페라,NaN
2,4829,첫사랑,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/03/27 ~ 2007/06/17,2007-03-27,2007-06-17,대학로 티오엠 1관,"조정석,홍광호,해이,전경수,이정섭,김성기,김인수...",만 6세이상,...,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"['시간', '조금', '수', '있다', '적', '동네', '해수', '오누이'...","b""S\xa2\xa0<\xeb\xad\xd5<\xd6\x93\xbe\xbd\x01\...",[ 0.01960865 0.02608391 -0.09305541 -0.089630...,"[-0.6486117243766785, 0.176923006772995, -1.25...",오페라,NaN,NaN
3,3322,미라클,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/06/01 ~ 2013/02/17,2007-06-01,2013-02-17,미라클 씨어터 1관,"신현묵,조호균,김고운,원혜지,남궁인,윤도훈,김학규...",만 12세이상,...,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"['인기', '그룹', '핫바', '멤버', '가수', '희다', '은', '날',...",b'^\xb5D>\xb63\xad=P\x8e >.\x97\x02\xbe\xee+\x...,[ 1.92098111e-01 8.45712870e-02 1.56792879e-...,"[0.5355554223060608, 0.6421719193458557, 0.540...",오페라,NaN,NaN
4,4424,대장금,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/05/26 ~ 2007/06/17,2007-05-26,2007-06-17,예술의전당 오페라극장,"최보영,안유진,김우형,김소현,이태원,류수화,손광업,한애리,한성식,김나윤,김일우,최윤...",만 7세이상,...,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"['어머니', '유언', '위해', '궁', '생각', '시가', '장금', '것'...",b'@\x8a\xa8=\x8e\xd7\xe0=\x1e\x10\x9b=N%\xf6\x...,[ 0.08229494 0.10978614 0.07571433 -0.120188...,"[-0.21826054155826569, 0.84274822473526, -0.04...",사랑,오페라,아이
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11982,150211,브레멘 음악대 부산,http://ticketimage.interpark.com/PlayDictionar...,어린이가족,2019/11/16 ~ 2019/11/17,2019-11-16,2019-11-17,신세계센텀시티 문화홀9층,NaN,24개월이상,...,NaN,집에서 키워지던 동물들이 주인의 천대를 받고쫓겨나거나 스스로의 자유를 위해 집을 나...,집 동물 주인 천대 스스로 자유 위해 집 친구 멋지다 악기 음악회 만들기 즐겁다 시...,"['집', '동물', '주인', '천대', '스스로', '자유', '위해', '집'...","b'e#^\xbc\xc5o\x10\xbd\x07,\x02>\xa4\xd4\x1c>\...",[-1.35582434e-02 -3.52628417e-02 1.27121076e-...,"[-0.8763076663017273, -0.3110712766647339, 0.3...",친구,예술,NaN
11983,150242,헤이지니 럭키강이 2 청주,http://ticketimage.interpark.com/PlayDictionar...,어린이가족,2019/12/28 ~ 2019/12/29,2019-12-28,2019-12-29,청주예술의전당 대공연장,NaN,24개월이상,...,NaN,딩동 마법의 세계로 가야할 택배상자가 지니강이 집으로 배달되었다신비로운 마법의 열쇠...,딩동 마법 세계 가야 택배 상자 지니 강 집 배달 신비롭다 마법 열쇠 지니 강 비밀...,"['딩동', '마법', '세계', '가야', '택배', '상자', '지니', '강'...","b""\xe1uF=`\x1c\xb3<\x01\x92\x93>\xfc&~\xbe|\xa...",[ 0.04845226 0.02186412 0.2882233 -0.248195...,"[-0.4505958557128906, 0.14335589110851288, 1.4...",오페라,마법,NaN
11984,150349,난타 대구,http://ticketimage.interpark.com/PlayDictionar...,퍼포먼스,2019/12/20 ~ 201

In [25]:
import ast

# 'synopsis_numpy_scale' 열의 NaN 값을 빈 리스트로 대체합니다
past_data['synopsis_numpy_scale'].fillna('[]', inplace=True)

/tmp/ipykernel_2390/312548407.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  past_data['synopsis_numpy_scale'].fillna('[]', inplace=True)


In [26]:
# 데이터프레임에서 synopsis_numpy_scale 열의 값을 파싱하여 리스트로 변환
past_data['synopsis_numpy_scale'] = past_data['synopsis_numpy_scale'].apply(ast.literal_eval)

/tmp/ipykernel_2390/1748585286.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  past_data['synopsis_numpy_scale'] = past_data['synopsis_numpy_scale'].apply(ast.literal_eval)


In [27]:
# StandardScaler를 사용하여 특성들을 표준 스케일링
scaler_past = StandardScaler()
past_data_scaled = scaler_past.fit_transform(np.vstack(past_data['synopsis_numpy_scale']))

In [28]:
past_data_scaled

array([[-0.68171345,  0.31413515, -0.20701817, ...,  0.25340229,
         0.00262065, -1.48198926],
       [ 0.03825666,  0.39366723, -0.70994244, ..., -0.66517038,
         0.12733542,  1.57094421],
       [-0.64976171,  0.1772706 , -1.25631835, ...,  0.14196588,
        -0.00419195,  0.4187336 ],
       ...,
       [ 0.29584333,  1.80983334,  0.62095799, ...,  0.14988127,
        -0.00425578, -0.84514081],
       [ 2.07094952, -1.67720276, -1.66907966, ..., -1.13705803,
         0.79483239,  1.4434709 ],
       [-0.62677678,  0.06158938, -0.92015862, ...,  0.03716495,
         0.24737914,  0.24233482]])

In [29]:
# KNN 모델 초기화
knn_model_past = NearestNeighbors(n_neighbors=7, metric='euclidean')
knn_model_past.fit(past_data_scaled)

NearestNeighbors(metric='euclidean', n_neighbors=7)

In [31]:
# 현재 데이터 로드
present_data = pd.read_csv('../present_data/Data5_present_final.csv')

In [32]:
# 데이터프레임에서 synopsis_numpy_scale 열의 값을 파싱하여 리스트로 변환
present_data['synopsis_numpy_scale'] = present_data['synopsis_numpy_scale'].apply(ast.literal_eval)

In [33]:
# StandardScaler를 사용하여 특성들을 표준 스케일링
scaler_present = StandardScaler()
present_data_scaled = scaler_present.fit_transform(np.vstack(present_data['synopsis_numpy_scale']))

In [34]:
# KNN 모델 초기화
knn_model_present = NearestNeighbors(n_neighbors=6, metric='euclidean')
knn_model_present.fit(present_data_scaled)

NearestNeighbors(metric='euclidean', n_neighbors=6)

In [35]:
# 과거 데이터에서 작품을 선택 (예: musical_id가 3885인 작품)
selected_work_index_past = past_data[past_data['musical_id'] == 3885].index[0]

In [36]:
# 선택한 작품과 유사한 작품 찾기
distances, indices = knn_model_present.kneighbors([present_data_scaled[selected_work_index_past]]) # 내주적으로 유클리디안 거리가 계산됨

In [37]:
# 최대 거리와 최소 거리 계산
max_distance = distances.max()
min_distance = distances.min()

# 정규화된 유사도 계산
normalized_distances = (1 - (distances - min_distance) / (max_distance - min_distance))

# 유사한 작품 출력
print("선택한 작품:")
print(past_data.iloc[selected_work_index_past]['title'])
print("")
print("유사한 작품들:")
for i in range(1, len(indices.flatten())):
    index = indices.flatten()[i]
    normalized_distance = normalized_distances.flatten()[i]
    title = present_data.iloc[index]['title']
    musical_id = present_data.iloc[index]['musical_id']
    print(f"유사한 작품: {title}, Musical ID: {musical_id}, 정규화된 유사도: {normalized_distance}")

선택한 작품:
실연남녀

유사한 작품들:
유사한 작품: 레미제라블, Musical ID: 190722, 정규화된 유사도: 1.0
유사한 작품: 옹알스, Musical ID: 192343, 정규화된 유사도: 0.10883971306498053
유사한 작품: 오페라의 유령, Musical ID: 183052, 정규화된 유사도: 0.04449668813101304
유사한 작품: 그림책 속 제주이야기, Musical ID: 190256, 정규화된 유사도: 0.040566956547089084
유사한 작품: 왓 이프, Musical ID: 186831, 정규화된 유사도: 0.0
